In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

# Initialize WebDriver (Ensure geckodriver is installed and in your PATH)
wd = webdriver.Firefox()

# Define the NBA stats URL and filename
end_year = 2025  # Change this to desired season
url = f"https://www.espn.com/nba/stats/player/_/season/{end_year}/seasontype/2"
filename = f'nba_player_stats_{end_year-1}-{end_year}.csv'

# Open the URL in WebDriver
wd.get(url)

# Pause to allow manual interaction with the "Show More" button
input("Press Enter after pressing 'Show More' button until the end...")

# Extract player names using Selenium
try:
    player_elements = wd.find_elements(By.XPATH, "//tr[contains(@class, 'Table__TR')]//td[2]//a")
    names = [element.text for element in player_elements if element.text]
    print(f"Number of player names extracted: {len(names)}")
except Exception as e:
    print("Error extracting player names:", e)
    names = []

# Extract tables using pandas
html_source = wd.page_source
tables = pd.read_html(html_source)

# Ensure tables exist
if len(tables) < 2:
    print("Error: Expected at least 2 tables but found", len(tables))
    wd.quit()
    exit()

# Extract player data
players = tables[0]
stats = tables[1]

# Drop 'RK' column if it exists
if 'RK' in players.columns:
    players = players.drop(columns=['RK'])

# Add extracted player names
players['Name'] = names

# Merge player and stats DataFrames
df = pd.concat([players, stats], axis=1)

# Display the final DataFrame
print(df.head())

# Save to CSV
df.to_csv(filename, index=False)
print(f"Data saved to {filename}")

# Close WebDriver
wd.quit()
